In [1]:
# NeMo's "core" package
import nemo
# NeMo's ASR collection - this collections contains complete ASR models and
# building blocks (modules) for ASR
import nemo.collections.asr as nemo_asr
import json
import librosa
import os
import pandas as pd
from pydub import AudioSegment

[NeMo W 2021-06-04 07:08:36 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text.AudioToCharDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2021-06-04 07:08:36 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text.AudioToBPEDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2021-06-04 07:08:36 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text.AudioLabelDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2021-06-04 07:08:36 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text._TarredAudioToTextDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2021-06-04 07:08:36 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text.TarredAudioToCharDataset'> is experimental,

# Making JSON Manifest Files for Nemo Training

In [2]:
# --- Building Manifest Files --- #

# Function to build a manifest
def build_manifest_df(df, manifest_path):
  with open(manifest_path, 'w') as fout:
    for index, row in df.iterrows():
      # mp3 to wav
      mp3sound = AudioSegment.from_mp3('../data/' + str(row.filepath).rsplit('/',1)[1])
      wav_path = ('../data/' + (str(row.filepath)).strip('.mp3') +'.wav').rsplit('/',1)[1]
      mp3sound.export('../data/' + wav_path, format="wav")
      print(wav_path)
      duration = librosa.core.get_duration(filename='../data/' + wav_path)
      # Write the metadata to the manifest
      metadata = {
               "audio_filepath": '../data/' + wav_path,
               "duration": duration,
               "text": row['Cleaned audio transcript']
                }
      json.dump(metadata, fout)
      fout.write('\n')

In [6]:
# Using audio data spreadsheet to build the manifests
df=pd.read_csv('..\\data\\running_records_audio_info.csv')

In [7]:
train_df = df[0:22].copy()
valid_df = df[23:36].copy()

In [8]:
build_manifest_df(train_df,'../data/train_manifest_all.json')
build_manifest_df(valid_df,'../data/test_manifest_all.json')

MezaDaniel.wav
HernandezJosue.wav
SuttonBryant (1).wav
PerezAlisson.wav
AscencioAlexis.wav
DomingoGenesis.wav
Carter-ShoafLondon.wav
RowellParis.wav
JohnsonAyden.wav
VestalBentley.wav
Seder Masen.wav
Guest Kaitlyn.wav
Brown Kaitlynn.wav
BrownJase.wav
GarciaMerlin.wav
MartinezMichael.wav
ChristieJeremy .wav
StreaterXavier.wav
ClarkSusan.wav
CollinsCole.wav
GreenLeslie.wav
Shannon, Sha_Mari.wav
Allen, Daniah.wav
HaleyMeya.wav
SmithElijah.wav
GhalyMina.wav
PatelVihaan.wav
BryantKimoni.wav
SuberKaden.wav
sierrasuarezangel.wav
StoutLa_elMetiMkhai.wav
SarkarAbhiraj.wav
MitchellTyler.wav
ErizaSarai.wav
NealElliann.wav


Out of Box Model Usage

In [9]:
#scr
# This line will download pre-trained QuartzNet15x5 model from NVIDIA's NGC cloud and instantiate it for you
quartznet = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="QuartzNet15x5Base-En")

[NeMo I 2021-06-04 07:11:04 cloud:56] Found existing object C:\Users\Carl\.cache\torch\NeMo\NeMo_1.0.0b3\QuartzNet15x5Base-En\00869f9c89b8393ca3de640e0c536bd2\QuartzNet15x5Base-En.nemo.
[NeMo I 2021-06-04 07:11:04 cloud:62] Re-using file from: C:\Users\Carl\.cache\torch\NeMo\NeMo_1.0.0b3\QuartzNet15x5Base-En\00869f9c89b8393ca3de640e0c536bd2\QuartzNet15x5Base-En.nemo
[NeMo I 2021-06-04 07:11:04 common:399] Instantiating model from pre-trained checkpoint


[NeMo W 2021-06-04 07:11:05 nemo_logging:349] c:\users\carl\.conda\envs\running_records\lib\site-packages\hydra\_internal\utils.py:632: UserWarning: 
    Config key 'cls' is deprecated since Hydra 1.0 and will be removed in Hydra 1.1.
    Use '_target_' instead of 'cls'.
    See https://hydra.cc/docs/next/upgrades/0.11_to_1.0/object_instantiation_changes
      warnings.warn(message=msg, category=UserWarning)
    
[NeMo W 2021-06-04 07:11:05 nemo_logging:349] c:\users\carl\.conda\envs\running_records\lib\site-packages\hydra\_internal\utils.py:577: UserWarning: 
    Field 'params' is deprecated since Hydra 1.0 and will be removed in Hydra 1.1.
    Inline the content of params directly at the containing node.
    See https://hydra.cc/docs/next/upgrades/0.11_to_1.0/object_instantiation_changes
      warnings.warn(category=UserWarning, message=msg)
    


[NeMo I 2021-06-04 07:11:05 features:241] PADDING: 16
[NeMo I 2021-06-04 07:11:05 features:258] STFT using torch
[NeMo I 2021-06-04 07:11:05 modelPT:257] Model EncDecCTCModel was successfully restored from C:\Users\Carl\.cache\torch\NeMo\NeMo_1.0.0b3\QuartzNet15x5Base-En\00869f9c89b8393ca3de640e0c536bd2\QuartzNet15x5Base-En.nemo.


In [10]:
# scr
quartznet.transcribe(paths2audio_files=['..\\data\\ClarkSusan.wav'])

[NeMo W 2021-06-04 07:12:30 nemo_logging:349] c:\users\carl\.conda\envs\running_records\lib\site-packages\torch\functional.py:516: UserWarning: stft will require the return_complex parameter be explicitly  specified in a future PyTorch release. Use return_complex=False  to preserve the current behavior or return_complex=True to return  a complex output. (Triggered internally at  ..\aten\src\ATen\native\SpectralOps.cpp:653.)
      normalized, onesided, return_complex)
    
[NeMo W 2021-06-04 07:12:30 nemo_logging:349] c:\users\carl\.conda\envs\running_records\lib\site-packages\torch\functional.py:516: UserWarning: The function torch.rfft is deprecated and will be removed in a future PyTorch release. Use the new torch.fft module functions, instead, by importing torch.fft and calling torch.fft.fft or torch.fft.rfft. (Triggered internally at  ..\aten\src\ATen\native\SpectralOps.cpp:590.)
      normalized, onesided, return_complex)
    


["omemmos old man moss woke up and fell out of the bed he fell on de fore he fell very hard the and he landed on his head old man moss was so called at the at he at she an pulled they blanket off the bed but he poll so hard at he tor deep blanket ald man mossset were onman masu masses i guess seed were very calld he wanted to find his socks first he looked under the bed next he looked under the hair but he could couldn't find his socks then he en tried to look in the closet but he but the doorknob stell off it blanded on his big toe old man mosse's toe hurt because the doornob fell on it his feet were cold because he couldn't his feet were called because he couldn't find his socks he was called all over becausehe he hep tortour his blanket he his head hurt because he sal on it so all man moss went back to bedo thank you so much for doing i"]

In [13]:
quartznet

EncDecCTCModel(
  (preprocessor): AudioToMelSpectrogramPreprocessor(
    (featurizer): FilterbankFeatures()
  )
  (encoder): ConvASREncoder(
    (encoder): Sequential(
      (0): JasperBlock(
        (mconv): ModuleList(
          (0): MaskedConv1d(
            (conv): Conv1d(64, 64, kernel_size=[33], stride=[2], padding=(16,), dilation=[1], groups=64, bias=False)
          )
          (1): MaskedConv1d(
            (conv): Conv1d(64, 256, kernel_size=(1,), stride=(1,), bias=False)
          )
          (2): BatchNorm1d(256, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        )
        (mout): Sequential(
          (0): ReLU()
          (1): Dropout(p=0.0, inplace=False)
        )
      )
      (1): JasperBlock(
        (mconv): ModuleList(
          (0): MaskedConv1d(
            (conv): Conv1d(256, 256, kernel_size=[33], stride=[1], padding=(16,), dilation=[1], groups=256, bias=False)
          )
          (1): MaskedConv1d(
            (conv): Conv1d(256, 256, ke

In [53]:
# Listen to the audio file we're transcribing:
from pydub.playback import play

audiofile = '..\\data\\ClarkSusan.wav'
AudioSegment.from_mp3(audiofile)

Training (in progress)

In [55]:
# Need to git clone NeMo only once. Don't include it in (our) git repository.
# !git clone https://github.com/NVIDIA/NeMo/

Cloning into 'NeMo'...


In [56]:
# --- Config Information ---#
try:
    from ruamel.yaml import YAML
except ModuleNotFoundError:
    from ruamel_yaml import YAML
#config_path = 'quartznet_15x5.yaml'

yaml = YAML(typ='safe')
with open('../NeMo/examples/asr/conf/quartznet_15x5.yaml') as f:
   params = yaml.load(f)
print(params)

{'name': 'QuartzNet15x5', 'model': {'sample_rate': 16000, 'repeat': 5, 'dropout': 0.0, 'separable': True, 'labels': [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'"], 'train_ds': {'manifest_filepath': '???', 'sample_rate': 16000, 'labels': [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'"], 'batch_size': 32, 'trim_silence': True, 'max_duration': 16.7, 'shuffle': True, 'is_tarred': False, 'tarred_audio_filepaths': None, 'tarred_shard_strategy': 'scatter'}, 'validation_ds': {'manifest_filepath': '???', 'sample_rate': 16000, 'labels': [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'"], 'batch_size': 32, 'shuffle': False}, 'test_ds': {'manifest_filepath': None, 'sample_rate': 16000, 'labels': [' ', 'a', 'b', 'c', 'd', 'e', 

In [57]:
import pytorch_lightning as pl

In [58]:
trainer = pl.Trainer(gpus=0, max_epochs=5)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


In [59]:
train_mainfest = '../data/train_manifest_all.json'
test_manifest = '../data/test_manifest_all.json'

In [60]:
params['model']

{'sample_rate': 16000,
 'repeat': 5,
 'dropout': 0.0,
 'separable': True,
 'labels': [' ',
  'a',
  'b',
  'c',
  'd',
  'e',
  'f',
  'g',
  'h',
  'i',
  'j',
  'k',
  'l',
  'm',
  'n',
  'o',
  'p',
  'q',
  'r',
  's',
  't',
  'u',
  'v',
  'w',
  'x',
  'y',
  'z',
  "'"],
 'train_ds': {'manifest_filepath': '???',
  'sample_rate': 16000,
  'labels': [' ',
   'a',
   'b',
   'c',
   'd',
   'e',
   'f',
   'g',
   'h',
   'i',
   'j',
   'k',
   'l',
   'm',
   'n',
   'o',
   'p',
   'q',
   'r',
   's',
   't',
   'u',
   'v',
   'w',
   'x',
   'y',
   'z',
   "'"],
  'batch_size': 32,
  'trim_silence': True,
  'max_duration': 16.7,
  'shuffle': True,
  'is_tarred': False,
  'tarred_audio_filepaths': None,
  'tarred_shard_strategy': 'scatter'},
 'validation_ds': {'manifest_filepath': '???',
  'sample_rate': 16000,
  'labels': [' ',
   'a',
   'b',
   'c',
   'd',
   'e',
   'f',
   'g',
   'h',
   'i',
   'j',
   'k',
   'l',
   'm',
   'n',
   'o',
   'p',
   'q',
   'r',
   

In [61]:
params['model']['train_ds']['max_duration'] = 500

In [63]:
from omegaconf import DictConfig
params['model']['train_ds']['manifest_filepath'] = '../data/train_manifest_all.json'
params['model']['validation_ds']['manifest_filepath'] = '../data/test_manifest_all.json'
first_asr_model = nemo_asr.models.EncDecCTCModel(cfg=DictConfig(params['model']), trainer=trainer)

[NeMo I 2021-03-04 09:03:49 collections:173] Dataset loaded with 21 files totalling 1.30 hours
[NeMo I 2021-03-04 09:03:49 collections:174] 1 files were filtered totalling 0.15 hours
[NeMo I 2021-03-04 09:03:49 collections:173] Dataset loaded with 13 files totalling 0.87 hours
[NeMo I 2021-03-04 09:03:49 collections:174] 0 files were filtered totalling 0.00 hours
[NeMo I 2021-03-04 09:03:49 features:241] PADDING: 16
[NeMo I 2021-03-04 09:03:49 features:258] STFT using torch


[NeMo W 2021-03-04 09:03:49 ctc_models:290] Could not load dataset as `manifest_filepath` was None. Provided config : {'manifest_filepath': None, 'sample_rate': 16000, 'labels': [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'"], 'batch_size': 32, 'shuffle': False}


In [64]:
# Start training!!!
trainer.fit(first_asr_model)

[NeMo I 2021-03-04 09:04:27 modelPT:597] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.8, 0.5]
        eps: 1e-08
        grad_averaging: False
        lr: 0.01
        weight_decay: 0.001
    )
[NeMo I 2021-03-04 09:04:27 lr_scheduler:562] Scheduler "<nemo.core.optim.lr_scheduler.CosineAnnealing object at 0x000001EF8ABC1E08>" 
    will be used during training (effective maximum steps = 5) - 
    Parameters : 
    (warmup_steps: null
    warmup_ratio: null
    min_lr: 0.0
    last_epoch: -1
    max_steps: 5
    )



  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConvASREncoder                    | 18.9 M
2 | decoder           | ConvASRDecoder                    | 29.7 K
3 | loss              | CTCLoss                           | 0     
4 | spec_augmentation | SpectrogramAugmentation           | 0     
5 | _wer              | WER                               | 0     


Validation sanity check: 0it [00:00, ?it/s]

ValueError: normalize_batch with `per_feature` normalize_type received a tensor of length 1. This will result in torch.std() returning nan